In [1]:
import subreddit_question_generation as sub_gen

# Generate Questions for Benchmark

In [ ]:
sub_gen.generate_for_benchmark()

# Generate Questions for Finetuning

In [6]:
sub_gen.generate_for_finetuning(10)

0it [00:01, ?it/s]


APIConnectionError: Connection error.

# Finetuning

In [1]:
import subreddit_finetuning as sub_fin

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from tqdm import tqdm

In [2]:
model = sub_fin.fine_tune_model()
model.eval()

extracting data


Map:   0%|          | 0/5723 [00:00<?, ? examples/s]

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

loading model


Map:   0%|          | 0/5723 [00:00<?, ? examples/s]

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


starting training


Step,Training Loss
10,1.927600
20,1.516400
30,1.390900
40,1.334100
50,1.440700
60,1.266200
70,1.329900
80,1.346900
90,1.253700
100,1.227200


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "/scratch/bchk/kbengani/reddit_sentiment_analysis/finetuned/checkpoint-3580/",
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2"
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("/scratch/bchk/kbengani/reddit_sentiment_analysis/finetuned/checkpoint-3580/", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
model.save_pretrained("subreddit_finetuned_model")

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    "/scratch/bchk/kbengani/reddit_sentiment_analysis/subreddit_finetuned_model/",
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2"
).to("cuda")

df = pd.read_csv('csv_files/subreddit_finetuning_benchmark.csv')

correct = 0
for i, row in tqdm(df.iterrows()):
    example_inputs = tokenizer([f"Question: {row['question']}\nSubreddit:"], return_tensors="pt").to(model.device)

    for _ in range(3):
        example_outputs = model.generate(
            **example_inputs, 
            max_length=200,
            temperature=0.9,
            do_sample=True,
            top_p=0.2,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id)
        output_text = tokenizer.decode(example_outputs[0])
        question = output_text.split("\n")[0].split("Question: ")[1]
        subreddit = output_text.split("\n")[1].split("Subreddit: ")[1]
        if subreddit == row["subreddit"]:
            correct += 1
            break

print(f"{correct / len(df) * 100}%")

580it [57:48,  5.98s/it]

28.448275862068968%
